#**Regulation Intake & Applicability Filter**

# Domain: Legal and Compliance
#Project Overview


AI-driven assistant for evaluating regulatory applicability, extracting obligations,  and assigning risk levels for organizations.


#Problem Statement & Domain Challenges
- Manual compliance checks are time-consuming and error-prone.
- Organizations operate across multiple regions and industries with complex regulations.
- Challenges include:
  - Evaluating multiple regulations per organization
  - Extracting enforceable obligations from contracts/policies
  - Assigning consistent risk levels
  - Presenting results in structured, audit-ready formats


#AI Models / Prompting Techniques Used
- Models:
  - `gemini-2.5-flash` → text reasoning, few-shot and one-shot prompts
  - `gpt-4.1-nano` → step-by-step Chain-of-Thought reasoning
  - `gpt-4o-mini-tts` → text-to-speech
  - `imagen-4.0-fast-generate-001` → decision tree visualization
- Techniques:
  - One-shot Prompting
  - Few-shot Prompting
  - Chain-of-Thought / Tree-of-Thought
  - Structured decision grids
  - JSON & tabular outputs


# Features & Capabilities Implemented
- Automated regulatory applicability: Ignore | Monitor | Act
- Obligation extraction & risk scoring
- Step-by-step reasoning for complex scenarios
- Optional TTS outputs
- Decision tree visualization
- JSON/tabular outputs for dashboards and audit reports


#Data Flow Architecture
1. Input: Regulation text / audio + organization profile
2. LLM Processing:
   - Template 1: One-shot applicability check
   - Template 2: Few-shot + structured decision grid
   - Template 3: Audio input + zero-shot analysis
3. Output:
   - Decision: Ignore | Monitor | Act
   - Optional TTS / transcription
   - JSON / tabular structured reasoning
4. Integration:
   - Compliance dashboards, reporting, or automation pipelines


## Use Case Description
As a compliance officer or regulatory analyst, you can use Template 1 to quickly assess whether a specific regulation applies to your organization.
Simply provide the regulation text and a brief profile of your organization (industry, regions, activities, data types).
The AI will reason using a single annotated example and return a structured decision: Ignore, Monitor, or Act, along with a concise justification.
This template is ideal for rapid single-regulation assessments where precedent examples are available.

## Prompt Template-1



In [ ]:
# =========================================================
# TASK 1: Regulation Intake & Applicability Filter
# TECHNIQUE: One-shot Prompting
# GOAL: Decide whether a regulation applies to an organization
# OUTPUT: Ignore | Monitor | Act
# =========================================================


def regulation_applicability_one_shot(regulation_text, organization_profile):
    """
    Determines regulatory applicability using a one-shot example.
    """

    # One-shot prompt with a single grounding example
    prompt = f"""
You are a regulatory compliance analyst.

Example:
Regulation:
Applies to financial institutions operating in the EU.

Organization Profile:
Industry: Healthcare
Region: United States

Decision: Ignore
Reason: Regulation applies only to EU financial institutions.

Now analyze the following.

Regulation Text:
{regulation_text}

Organization Profile:
{organization_profile}

Return ONLY in this format:
Decision: Ignore | Monitor | Act
Reason: One sentence justification
"""

    # Call the LLM
    response = client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=[{"role": "user", "content": prompt}],
        temperature=0  # Ensures consistent decisions
    )

    # Extract model response
    return response.choices[0].message.content


##Sample Input

In [ ]:
sample_regulation_text = """
This regulation applies to organizations that collect, store, or process
personal data of individuals located in the European Union. Covered data
includes names, email addresses, IP addresses, and online identifiers.
Organizations must implement appropriate technical and organizational
measures to ensure data protection.
"""

sample_organization_profile = """
Industry: Software as a Service (SaaS)
Regions of Operation: United States, European Union
Business Model: Cloud-based subscription platform
Data Collected: Names, emails, IP addresses, usage analytics
Data Storage: Cloud servers hosted in the United States
"""


##Sample Output

In [ ]:
output = regulation_applicability_one_shot(
    regulation_text=sample_regulation_text,
    organization_profile=sample_organization_profile
)

print(output)


Decision: Act
Reason: The organization operates in the European Union and collects personal data, including names, emails, and IP addresses, from individuals located there, which falls under the scope of this regulation.


------------------------------------------

##TEMPLATE 2
#Use Case Description
This allows regulatory analysts to evaluate regulatory applicability using multiple example scenarios.
By providing several grounding examples and your organization profile, the AI uses few-shot reasoning and a structured decision grid to deliver a well-informed decision.
Output includes Target Region, Target Industry, Covered Activity, Organization Match, along with a final decision (Ignore, Monitor, Act) and a brief justification.
This template is suited for complex regulations or organizations operating across multiple regions or industries.

In [ ]:
# =========================================================
# TASK 1: Regulation Intake & Applicability Filter
# TECHNIQUE: Few-shot + Structured Decision Grid
# MODALITY: Tabular reasoning + constraints
# OUTPUT: Ignore | Monitor | Act
# =========================================================


def regulation_applicability_structured(regulation_text, organization_profile):
    """
    Determines regulatory applicability using examples
    and a structured decision framework.
    """

    prompt = f"""
You are a regulatory compliance analyst.

Use the decision grid below to make your determination.

Decision Rules:
- If regulation clearly applies → Act
- If applicability depends on unclear scope → Monitor
- If regulation clearly does not apply → Ignore

Example 1:
Regulation Scope: EU financial institutions
Org Region: United States
Org Industry: Healthcare
Decision: Ignore

Example 2:
Regulation Scope: EU personal data processing
Org Region: EU + US
Org Activity: Processes customer data
Decision: Act

Example 3:
Regulation Scope: APAC manufacturing firms >10,000 employees
Org Employees: 2,000
Org Region: Europe
Decision: Ignore

Now complete the grid for the following.

Regulation Text:
{regulation_text}

Organization Profile:
{organization_profile}

Decision Grid:
- Target Region:
- Target Industry:
- Covered Activity:
- Organization Match (Yes / Partial / No):

Return ONLY:
Decision: Ignore | Monitor | Act
Reason: One sentence justification
"""

    response = client.chat.completions.create(
        model="gemini-2.5-flash",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )

    return response.choices[0].message.content


#Sample Input

In [ ]:
sample_regulation_text = """
This regulation applies to organizations offering digital services
to individuals located in the European Union and processing personal
data such as names, emails, or online identifiers.
"""

sample_organization_profile = """
Industry: SaaS
Regions: United States, European Union
Services: Cloud-based digital platform
Data Processed: Customer names, emails, usage data
"""


#Sample Output

In [ ]:
output = regulation_applicability_structured(
    regulation_text=sample_regulation_text,
    organization_profile=sample_organization_profile
)

print(output)


Decision Grid:
- Target Region: European Union
- Target Industry: Organizations offering digital services
- Covered Activity: Processing personal data (names, emails, online identifiers)
- Organization Match (Yes / Partial / No): Yes

Decision: Act
Reason: The organization clearly offers digital services to individuals in the EU and processes relevant personal data, directly meeting all regulatory criteria.


#Template-3
#Use Case Description
Template 3 enables compliance teams to process audio-recorded regulations or policy statements.
Users provide an audio file of the regulation and the organization profile. The AI first transcribes the audio and then performs zero-shot reasoning to determine applicability.
The output includes:

Transcribed regulation text

Step-by-step analysis

Final decision (Ignore, Monitor, Act)
This template is especially useful for processing spoken regulations, meetings, or briefings without needing manual transcription.

In [ ]:
# =========================================================
# TASK 1: Regulation Intake & Applicability Filter
# TECHNIQUE: Zero-shot + Structured Decision Grid
# MODALITY: Tabular reasoning + constraints
# OUTPUT: Ignore | Monitor | Act
# =========================================================

result = regulation_applicability_audio_first(
    audio_file_path="regulation_audio.wav",
    organization_profile="""
Industry: SaaS
Regions: United States, European Union
Activities: Digital platform
Data: Names, emails, IP addresses
"""
)

print("Transcription:\n", result["transcribed_regulation"])
print("\nAnalysis:\n", result["analysis"])
print("\nFinal Decision:\n", result["final_decision"])


#Sample Input

In [ ]:
sample_regulation_text = """
This regulation applies to organizations offering digital services
to individuals located in the European Union and processing personal
data such as names, emails, or online identifiers.
"""

sample_organization_profile = """
Industry: SaaS
Regions: United States, European Union
Services: Cloud-based digital platform
Data Processed: Customer names, emails, usage data
"""


#Sample Output

Transcription:
 This regulation applies to organizations that provide digital services to individuals located in the European Union, and that process personal data such as names, email addresses, IP addresses, or online identifiers.

Analysis:
 Geography: Yes
Industry: Yes
Activity: Yes

Final Decision:
 Decision: Act  
Reason: All three dimensions are Yes, which requires action according to the rules.

#Summary & Reusability Notes
- Notebook provides a modular AI compliance framework.
- Templates are self-contained, reusable for different regulations or organizations.
- Outputs (JSON, decision trees, audio) allow integration into dashboards or reports.
- Scalable for multi-region, multi-industry compliance assessments.
